# Dataset preprocessing

In [1]:
import os

import pandas as pd

from sarcsdet.utils.mark_ling_feat import (
    funny_marks_feature, interjections_feature, get_popular_items
)
from sarcsdet.utils.preprocess_data import Preprocessor

### Get data

In [2]:
data_path = '../data/Twitter'

In [3]:
df = pd.read_csv(os.path.join(data_path, 'twitter.csv'), index_col='id')

In [4]:
# source: twitter = 40000
df['source'] = 40000

# submitted_by: twitter = 16000
df['submitted_by'] = 16000

# target: twitter = all sarcastic/ironic
df['target'] = 1

In [5]:
pd.set_option('display.max_colwidth', df.shape[0] + 1)
df.head()

,tweet,rating,comments,source,submitted_by,target
id,,,,,,
1369000399764619271,А уже можно высказываться о песне Манижи? Или надо подождать официальную рецензию Никсель Пиксель?,4724,81,40000,16000,1
1381504875444498435,"Выигрышеая стратегия для мужика при высоком возрасте. дожития - в 47 лет завести 100 детей. А лучше - 200. У женщины такой стратегии, правда, нет, ...",0,0,40000,16000,1
1384538174861492228,"А в итоге окажется, что экономика лженаука и всё это время правда можно было напечатать столько денег, чтобы всем хватило",9324,51,40000,16000,1
1383607877219151882,Дружба между коллегами невозможна. Кто-то кому-то точно хочет подсунуть работу.,4606,15,40000,16000,1
1382490754468184068,"Майор Гром такой классный. Он за справедливость и пытает только плохих :) А анонимность это плохо, ведь она нужна террористам :( А народ кровожадны...",1553,49,40000,16000,1


### Text tokenization

In [6]:
preprocessor = Preprocessor('ru')

[nltk_data] Downloading package stopwords to /home/kate/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
df['tweet_tokenized'] = df.tweet.apply(preprocessor.preprocess_text)

In [8]:
df = df[df['tweet_tokenized'].apply(lambda x: len(x) > 10)]

### Extract linguistic features

In [9]:
df['tweet'] = pd.Series(df['tweet'], dtype='str')
df['tweet_tokenized'] = pd.Series(df['tweet_tokenized'], dtype='str')

df['rating'] = pd.Series(df['rating'], dtype='int')
df['comments'] = pd.Series(df['comments'], dtype='int')
df['source'] = pd.Series(df['source'], dtype='int')
df['submitted_by'] = pd.Series(df['submitted_by'], dtype='int')
df['target'] = pd.Series(df['target'], dtype='int')

### Add Linguistic Features

In [10]:
df['exclamation'] = df.tweet.apply(lambda s: s.count('!'))
df['question'] = df.tweet.apply(lambda s: s.count('?'))
df['dotes'] = df.tweet.apply(lambda s: s.count('...'))
df['quotes'] = df.tweet.apply(lambda s: s.count("\"") + s.count("\'"))

In [11]:
df['funny_mark'] = df.tweet.apply(funny_marks_feature)

In [12]:
df['interjections'] = df.tweet.apply(interjections_feature)

### Rename columns

In [13]:
df.rename(
    columns={
        'tweet': 'quote', 'tweet_tokenized': 'quote_tokenized', 
        'comments': 'comments_count'
    }, inplace=True
)

### Save dataset

In [14]:
df.to_csv(os.path.join(data_path, 'twitter_tokenized.csv'))